In [1]:
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, KernelDensity
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
import pandas as pd

In [2]:
#Load dataset into Pandas Dataframe
df3 = pd.read_csv('letter-recognition.data')

# Letter Dataset P.2

In [3]:
#Rename Column
letter_df = df3
letter_df = letter_df.rename(columns={'T': 'Letter'})
#Initialize Column
letter_df['Class'] = 0
letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M']
#For each row in the dataframe, if the alphabet letter in that row is not in the list above,
# set the class of that data point to 0. Otherwise, it's 1.
for index, row in letter_df.iterrows():
    if row['Letter'] in letters:
        letter_df.at[index, 'Class'] = 1
letter_df = letter_df.drop(['Letter'], axis = 1)
#Converting dataframe into array.
letter_arr = letter_df.to_numpy()

In [4]:
#Accuracy function. Compares predicted labels vs actual labels and counts accuracy.
def prediction(predictions, Y_given):
    wrong = 0
    counter = 0
    for test, train in zip(predictions, Y_given):
        if test == train:
            wrong = wrong
        else:
            wrong = wrong + 1
        counter = counter + 1
    accuracy = 1 - (wrong/counter)
    return accuracy

# SVM

In [5]:
#Three lists to store training accuracy for each trial, test accuracy for each trial, and best parameters for each trial.
SVM_train_accuracy = []
SVM_test_accuracy = []
SVM_best_params = []

#For each Trial
for i in range(3):
    data_svm = shuffle(letter_arr)
#Data splitting, train-test-split.
    X_svm = data_svm[:, 0:-1]
    Y_svm = data_svm[:, -1]
    X_train_svm, X_test_svm, Y_train_svm, Y_test_svm = train_test_split(X_svm, Y_svm, test_size = 14999/19999, random_state=42)
#Scaling training data using StandardScaler
    scaler_svm = preprocessing.StandardScaler().fit(X_train_svm)
    X_train_svm = scaler_svm.transform(X_train_svm)
#Param-grid, setting SVM class,grid search parameters, and fitting training data.
    parameters = [{'kernel': ['rbf'], 'gamma': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0, 2.0], 'C': [10**-7, 10**-6, 10**-5,
                                                                                                     10**-4, 10**-3, 10**-2,
                                                                                                     10**-1]},
                  {'kernel': ['poly'], 'degree': [2, 3], 'C': [10**-7, 10**-6, 10**-5, 10**-4, 10**-3, 10**-2, 10**-1]},
                  {'kernel': ['linear'], 'C': [10**-7, 10**-6, 10**-5, 10**-4, 10**-3, 10**-2, 10**-1]}]
    svc = svm.SVC(gamma = 'auto')
    grid_search_svm = GridSearchCV(svc, parameters, cv = 5, error_score = np.nan)
    grid_search2_svm = grid_search_svm.fit(X_train_svm, Y_train_svm)
    best_params_svm = grid_search2_svm.best_params_
#Printing and storing best params into list.
    print("Best params: {}".format(best_params_svm))
    SVM_best_params.append('Trial ' + str(i + 1) + ': ' + str(best_params_svm))
#Using best_estimator to predict training data labels.
#Using prediction function to calculate training accuracy.
#Printing and storing training accuracies into list.
    train_predictions_svm = grid_search2_svm.best_estimator_.predict(X_train_svm) #.best_estimator_
    train_accuracy_svm = prediction(train_predictions_svm, Y_train_svm)
    print("Train accuracy: {}".format(train_accuracy_svm))
    SVM_train_accuracy.append('Trial ' + str(i + 1) + ': ' + str(train_accuracy_svm))
#Scaling testing data with my training data scaler. This ensures training and testing data are scaled the same.
#Using prediction function to calculate testing accuracy.
#Printing and storing test accuracy into list.
    X_test_svm = scaler_svm.transform(X_test_svm)
    test_predictions_svm = grid_search2_svm.best_estimator_.predict(X_test_svm) #.best_estimator_
    test_accuracy_svm = prediction(test_predictions_svm, Y_test_svm)
    print("Test accuracy: {}".format(test_accuracy_svm))
    SVM_test_accuracy.append('Trial ' + str(i + 1) + ': ' + str(test_accuracy_svm))

Best params: {'C': 0.1, 'gamma': 0.5, 'kernel': 'rbf'}
Train accuracy: 0.9566
Test accuracy: 0.8923261550770052
Best params: {'C': 0.1, 'gamma': 0.5, 'kernel': 'rbf'}
Train accuracy: 0.9672
Test accuracy: 0.9193279551970132
Best params: {'C': 0.1, 'gamma': 0.5, 'kernel': 'rbf'}
Train accuracy: 0.9502
Test accuracy: 0.8862590839389293


# kNN

In [6]:
#Three lists to store training accuracy for each trial, test accuracy for each trial, and best parameters for each trial.
kNN_train_accuracy = []
kNN_test_accuracy = []
kNN_best_params = []

#Setting K parameters
p = (np.linspace(1,500,25))
p = p.astype('int64')

#For each trial
for i in range(3):
    data_knn = shuffle(letter_arr)
#Data splitting, train-test-split
    X_knn = data_knn[:, 0:-1]
    Y_knn = data_knn[:, -1]
    X_train_knn, X_test_knn, Y_train_knn, Y_test_knn = train_test_split(X_knn, Y_knn, test_size = 14999/19999, random_state=42)
# Scaling training data using StandardScaler()
    scaler_knn = preprocessing.StandardScaler().fit(X_train_knn)
    X_train_knn = scaler_knn.transform(X_train_knn)
#Param-grid, initializing kNN class, grid search parameters and fitting training data.
# Printing and storing best parameters in list.
    params = [{'weights' : ['uniform', 'distance'], 'metric' : ['minkowski'],'n_neighbors': p}]
    neighbor = KNeighborsClassifier()
    grid_search_knn = GridSearchCV(neighbor, params, cv=5, error_score = np.nan)
    grid_search_knn2 = grid_search_knn.fit(X_train_knn, Y_train_knn)
    best_params_knn = grid_search_knn2.best_params_
    print("Best params: {}".format(best_params_knn))
    kNN_best_params.append('Trial ' + str(i + 1) + ': ' + str(best_params_knn))
#Using best_estimator to predict training data labels.
#Using prediction function to calculate training accuracy
#Printing and storing training accuracy into list.
    train_predictions_knn = grid_search_knn2.best_estimator_.predict(X_train_knn)
    train_accuracy_knn = prediction(train_predictions_knn, Y_train_knn)
    print("Train accuracy: {}".format(train_accuracy_knn))
    kNN_train_accuracy.append('Trial ' + str(i + 1) + ': ' + str(train_accuracy_knn))
#Scaling testing data with my training data scaler. This ensures training and testing data are scaled the same.
#Using prediction function to calculate testing accuracy.
#Printing and storing test accuracy into list.
    X_test_knn = scaler_knn.transform(X_test_knn)
    test_predictions_knn = grid_search_knn2.best_estimator_.predict(X_test_knn)
    test_accuracy_knn = prediction(test_predictions_knn, Y_test_knn)
    print("Test accuracy: {}".format(test_accuracy_knn))
    kNN_test_accuracy.append('Trial ' + str(i + 1) + ': ' + str(test_accuracy_knn))

Best params: {'metric': 'minkowski', 'n_neighbors': 1, 'weights': 'uniform'}
Train accuracy: 1.0
Test accuracy: 0.950530035335689
Best params: {'metric': 'minkowski', 'n_neighbors': 1, 'weights': 'uniform'}
Train accuracy: 1.0
Test accuracy: 0.9535302353490233
Best params: {'metric': 'minkowski', 'n_neighbors': 1, 'weights': 'uniform'}
Train accuracy: 1.0
Test accuracy: 0.9537302486832455


# Random Forest

In [9]:
#Three lists to store training accuracy for each trial, test accuracy for each trial, and best parameters for each trial.
rf_train_accuracy = []
rf_test_accuracy = []
rf_best_params = []

#For each trial
for i in range(3):
    data_rf = shuffle(letter_arr)
#Data splitting, train-test-split
    X_rf = data_rf[:, 0:-1]
    Y_rf = data_rf[:, -1]
    X_train_rf, X_test_rf, Y_train_rf, Y_test_rf = train_test_split(X_rf, Y_rf, test_size = 14999/19999, random_state=42)
#Param-grid, initialize random forest class, and fitting training data with grid search.
    params = [{'n_estimators' : [1024], 'max_features' : [1, 2, 4, 6, 8, 12, 16, 20]}]
    forest = RandomForestClassifier()
    grid_search_rf = GridSearchCV(forest, params, cv=5, error_score = np.nan)
    grid_search_rf2 = grid_search_rf.fit(X_train_rf, Y_train_rf)
#Printing and storing best params into list.
    best_params_rf = grid_search_rf2.best_params_
    print("Best params: {}".format(best_params_rf))
    rf_best_params.append('Trial ' + str(i + 1) + ': ' + str(best_params_rf))
#Using best_estimator to predict training data.
#Storing and printing training accuracy into list.
#Using defined prediction function to calculate total accuracy.
    train_predictions_rf = grid_search_rf2.best_estimator_.predict(X_train_rf)
    train_accuracy_rf = prediction(train_predictions_rf, Y_train_rf)
    print("Train accuracy: {}".format(train_accuracy_rf))
    rf_train_accuracy.append('Trial ' + str(i + 1) + ': ' + str(train_accuracy_rf))
#Predicting Test data with best_estimator.predict.
#Using prediction function to calculate accuracy.
#Printing and storing Test accuracy into list.
    test_predictions_rf = grid_search_rf2.best_estimator_.predict(X_test_rf)
    test_accuracy_rf = prediction(test_predictions_rf, Y_test_rf)
    print("Test accuracy: {}".format(test_accuracy_rf))
    rf_test_accuracy.append('Trial ' + str(i + 1) + ': ' + str(test_accuracy_rf))

c:\users\zingb\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\model_selection\_validation.py:545: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: max_features must be in (0, n_features]

  FitFailedWarning)
c:\users\zingb\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\model_selection\_validation.py:545: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: max_features must be in (0, n_features]

  FitFailedWarning)
c:\users\zingb\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\model_selection\_validation.py:545: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: max_features must be in (0, n_features]

  FitFailedWarning)
c:\users\zingb\appdata\local\progr

Best params: {'max_features': 4, 'n_estimators': 1024}
Train accuracy: 1.0
Test accuracy: 0.9449963330888725


c:\users\zingb\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\model_selection\_validation.py:545: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: max_features must be in (0, n_features]

  FitFailedWarning)
c:\users\zingb\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\model_selection\_validation.py:545: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: max_features must be in (0, n_features]

  FitFailedWarning)
c:\users\zingb\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\model_selection\_validation.py:545: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: max_features must be in (0, n_features]

  FitFailedWarning)
c:\users\zingb\appdata\local\progr

Best params: {'max_features': 4, 'n_estimators': 1024}
Train accuracy: 1.0
Test accuracy: 0.9455963730915394


c:\users\zingb\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\model_selection\_validation.py:545: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: max_features must be in (0, n_features]

  FitFailedWarning)
c:\users\zingb\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\model_selection\_validation.py:545: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: max_features must be in (0, n_features]

  FitFailedWarning)
c:\users\zingb\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\model_selection\_validation.py:545: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: max_features must be in (0, n_features]

  FitFailedWarning)
c:\users\zingb\appdata\local\progr

Best params: {'max_features': 2, 'n_estimators': 1024}
Train accuracy: 1.0
Test accuracy: 0.9453963597573172
